In [28]:
import tensorflow as tf

In [29]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    "Dataset/train",
    image_size = (224,224),
    batch_size = 8,
    label_mode = "binary"
)
val_ds = tf.keras.utils.image_dataset_from_directory(
    "Dataset/validation",
    image_size = (224,224),
    batch_size = 8,
    label_mode = "binary"
)
test_ds = tf.keras.utils.image_dataset_from_directory(
    "Dataset/test",
    image_size = (224,224),
    batch_size = 8,
    label_mode = "binary"
)

Found 11633 files belonging to 2 classes.
Found 2400 files belonging to 2 classes.
Found 2400 files belonging to 2 classes.


In [30]:
data_aug = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomZoom(0.1),
    tf.keras.layers.RandomContrast(0.2),
    tf.keras.layers.GaussianNoise(0.1)
])

train_ds = train_ds.map(lambda x,y: (data_aug(x, training=True), y))

In [31]:
train_ds = train_ds.map(lambda x,y: (tf.keras.applications.xception.preprocess_input(x), y))
val_ds = val_ds.map(lambda x,y: (tf.keras.applications.xception.preprocess_input(x), y))
test_ds = test_ds.map(lambda x,y: (tf.keras.applications.xception.preprocess_input(x), y))

In [ ]:
base_model = tf.keras.applications.xception.Xception(
    weights = "imagenet",
    include_top = False,
    input_shape = (224,224,3)
)

base_model.trainable = False


In [37]:
x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
x = tf.keras.layers.BatchNormalization()(x)

x = tf.keras.layers.Dense(512, activation="relu")(x) 
x = tf.keras.layers.Dropout(0.5)(x)

x = tf.keras.layers.Dense(256, activation="relu")(x)
x = tf.keras.layers.Dropout(0.5)(x)

output = tf.keras.layers.Dense(1, activation="sigmoid")(x)

model = tf.keras.Model(inputs=base_model.input, outputs=output)

In [38]:
model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss = tf.keras.losses.BinaryCrossentropy(label_smoothing=0.1),
    metrics = ["accuracy", tf.keras.metrics.AUC(name="auc")]
)

In [39]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor = "val_loss",
    patience = 3,
    restore_best_weights = True
)

In [40]:
model.fit(train_ds, validation_data=val_ds, epochs=10, callbacks=[early_stopping])

Epoch 1/10


I0000 00:00:1769276301.972359   68712 service.cc:152] XLA service 0x75f8880d33b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1769276301.972585   68712 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 4050 Laptop GPU, Compute Capability 8.9
2026-01-24 17:38:22.141843: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1769276303.262697   68712 cuda_dnn.cc:529] Loaded cuDNN version 91800
2026-01-24 17:38:30.515576: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-01-24 17:38:30.724187: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup e

1453/1455 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.6359 - auc: 0.6815 - loss: 0.8215

2026-01-24 17:39:33.880814: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-01-24 17:39:34.063435: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-01-24 17:39:34.250071: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-01-24 17:39:34.833668: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-01-24 17:39:35.017854: E external/local_xla/xla/stream_

1455/1455 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 0.6360 - auc: 0.6816 - loss: 0.8214

2026-01-24 17:39:44.213129: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1072', 64 bytes spill stores, 64 bytes spill loads



1455/1455 ━━━━━━━━━━━━━━━━━━━━ 97s 53ms/step - accuracy: 0.6758 - auc: 0.7334 - loss: 0.6843 - val_accuracy: 0.6800 - val_auc: 0.7472 - val_loss: 0.6144
Epoch 2/10
1455/1455 ━━━━━━━━━━━━━━━━━━━━ 57s 39ms/step - accuracy: 0.7515 - auc: 0.8305 - loss: 0.5548 - val_accuracy: 0.6971 - val_auc: 0.7736 - val_loss: 0.6081
Epoch 3/10
1455/1455 ━━━━━━━━━━━━━━━━━━━━ 57s 39ms/step - accuracy: 0.7800 - auc: 0.8600 - loss: 0.5264 - val_accuracy: 0.7054 - val_auc: 0.7788 - val_loss: 0.6044
Epoch 4/10
1455/1455 ━━━━━━━━━━━━━━━━━━━━ 57s 39ms/step - accuracy: 0.7911 - auc: 0.8745 - loss: 0.5115 - val_accuracy: 0.6996 - val_auc: 0.7918 - val_loss: 0.6074
Epoch 5/10
1455/1455 ━━━━━━━━━━━━━━━━━━━━ 56s 39ms/step - accuracy: 0.8111 - auc: 0.8911 - loss: 0.4920 - val_accuracy: 0.7129 - val_auc: 0.7901 - val_loss: 0.6200
Epoch 6/10
1455/1455 ━━━━━━━━━━━━━━━━━━━━ 57s 39ms/step - accuracy: 0.8187 - auc: 0.8987 - loss: 0.4827 - val_accuracy: 0.7196 - val_auc: 0.7936 - val_loss: 0.6144


In [41]:
base_model.trainable = True

for layer in base_model.layers[:-10]:
    layer.trainable = False

In [42]:
model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss = tf.keras.losses.BinaryCrossentropy(label_smoothing=0.1),
    metrics = ["accuracy", tf.keras.metrics.AUC(name="auc")]
)

In [43]:
model.fit(train_ds, validation_data=val_ds, epochs=10, callbacks=[early_stopping])

Epoch 1/10
1455/1455 ━━━━━━━━━━━━━━━━━━━━ 82s 46ms/step - accuracy: 0.7621 - auc: 0.8449 - loss: 0.5443 - val_accuracy: 0.7633 - val_auc: 0.8458 - val_loss: 0.5419
Epoch 2/10
1455/1455 ━━━━━━━━━━━━━━━━━━━━ 57s 39ms/step - accuracy: 0.8374 - auc: 0.9178 - loss: 0.4595 - val_accuracy: 0.7921 - val_auc: 0.8832 - val_loss: 0.5076
Epoch 3/10
1455/1455 ━━━━━━━━━━━━━━━━━━━━ 58s 40ms/step - accuracy: 0.8773 - auc: 0.9475 - loss: 0.4141 - val_accuracy: 0.8229 - val_auc: 0.9030 - val_loss: 0.4799
Epoch 4/10
1455/1455 ━━━━━━━━━━━━━━━━━━━━ 57s 39ms/step - accuracy: 0.8914 - auc: 0.9592 - loss: 0.3927 - val_accuracy: 0.8217 - val_auc: 0.9084 - val_loss: 0.4766
Epoch 5/10
1455/1455 ━━━━━━━━━━━━━━━━━━━━ 57s 39ms/step - accuracy: 0.9049 - auc: 0.9683 - loss: 0.3737 - val_accuracy: 0.8021 - val_auc: 0.9119 - val_loss: 0.5370
Epoch 6/10
1455/1455 ━━━━━━━━━━━━━━━━━━━━ 56s 38ms/step - accuracy: 0.9182 - auc: 0.9740 - loss: 0.3585 - val_accuracy: 0.8383 - val_auc: 0.9188 - val_loss: 0.4539
Epoch 7/10
1455/

In [44]:
model.save("dfd_model.keras")